In [1]:
import pandas as pd
import myFunction as mf
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import auc, roc_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
from sklearn.externals import joblib
#import joblib
from xgboost.sklearn import XGBClassifier
import random
from sklearn.utils import shuffle
from scipy import stats
import myFunction as mf
from sklearn.neural_network import MLPClassifier
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None) #show all columns
pd.set_option('max_colwidth',None)

In [2]:
#train test data split
train_variables=['gender', 'age', 'vasopressin',  'urineoutput', 'heartrate_mean', 
                 'sysbp_mean', 'diasbp_mean', 'resprate_mean', 'tempc_mean', 'spo2_mean',
                 'baseexcess_mean', 'totalco2_mean', 'calcium_mean', 'lactate_mean', 
       'pco2_mean', 'ph_mean', 'po2_mean',
       'coronary heart disease.csv', 'diabetes.csv', 'family history of stroke.csv', 
       'creatinine.csv_mean',
       'glucose.csv_mean', 'platelet.csv_mean', 'potassium.csv_mean',
       'sodium.csv_mean', 'urea nitrogen.csv_mean', 'WBC.csv_mean', 'aniongap',
       'bicarbonate', 'hematocrit', 'hemoglobin', 'ptt', 'inr', 'pt', 'BMI']
outcome_variables=['liver_dysfunction','thrombocytopenia','hospital_mortality',
                  'septic_shock']


data=pd.read_csv('result/trainAndTestV6.csv')
split_percentage=0.8
sub_data=data[list(train_variables)+outcome_variables]
train=sub_data[:int(len(data)*split_percentage)]
test=sub_data[int(len(data)*split_percentage):]

FileNotFoundError: [Errno 2] No such file or directory: 'result/trainAndTestV6.csv'

In [ ]:
data.describe()

In [ ]:
# XGBoost
report=[]
count=0
list2=[]
for i in range(500,9001,100):
    list2.append(i)
list2
for time,scale_pos_weight in enumerate([0.01,0.02,0.03,0.04,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80
                                        ,90,100,200,300,400,500]+list2):
    for i in outcome_variables:
        count=count+1
        parameters=[{'n_estimators': [30,60], 
                'max_depth': [30,60],
                'reg_lambda':[0.1,1,10],
                'gamma':[0.1,1,10],
                'scale_pos_weight':[scale_pos_weight]}]
        model=GridSearchCV(XGBClassifier(n_jobs=-1), parameters, cv=5)
        model.fit(train[train_variables], train[i])
        y_score=model.predict_proba(test[train_variables])[:,1]
        fpr, tpr, _ = roc_curve(test[i], y_score)
        AUROC=auc(fpr, tpr)
        trainS=model.score(train[train_variables], train[i])
        testS=model.score(test[train_variables], test[i])
        f1S=metrics.f1_score(test[i], model.predict(test[train_variables]))
        preS=metrics.precision_score(test[i], model.predict(test[train_variables]))
        recS=metrics.recall_score(test[i], model.predict(test[train_variables]))
        AUROC=auc(fpr, tpr)
        feature_importance=pd.DataFrame({'column':train_variables, 'values':model.best_estimator_.feature_importances_})
        feature_importance.sort_values(by='values', axis=0, ascending=False, inplace=True)
        report.append([trainS, testS, f1S, preS,recS,AUROC,model.best_params_, i,
                      feature_importance.iloc[0].values,feature_importance.iloc[1].values,feature_importance.iloc[2].values,
                      feature_importance.iloc[3].values,feature_importance.iloc[4].values,scale_pos_weight])
        print(i,':')
        print('scale:', scale_pos_weight)
        print(trainS,'-',testS,'-',f1S,'-',preS,'-',recS,'-',AUROC,'-',model.best_params_)
        print('---------------------------------')
    res=pd.DataFrame(report)
    res.to_csv('result/trained models/XGB_models/report.csv')
    #joblib.dump(model, 'result/trained models/XGB_models/'+i+'_XGB.model')

In [ ]:
#logistic regression


report=[]
count=0
for time,scale_pos_weight in enumerate([0.01,0.02,0.03,0.04,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80
                                        ,90,100,200,300,400,500]+list2):
    for j, i in enumerate(outcome_variables):
        count=count+1
        parameters=[{'C':[0.01,0.1,1,10],
                    'class_weight':['balanced'],
                    'max_iter':[5]}]
        model=GridSearchCV(LogisticRegression(n_jobs=-1), parameters, cv=5)
        model.fit(train[train_variables], train[i])
        y_score=model.predict_proba(test[train_variables])[:,1]
        fpr, tpr, _ = roc_curve(test[i], y_score)
        AUROC=auc(fpr, tpr)
        trainS=model.score(train[train_variables], train[i])
        testS=model.score(test[train_variables], test[i])
        f1S=metrics.f1_score(test[i], model.predict(test[train_variables]))
        preS=metrics.precision_score(test[i], model.predict(test[train_variables]))
        recS=metrics.recall_score(test[i], model.predict(test[train_variables]))
        AUROC=auc(fpr, tpr)
        #feature_importance=pd.DataFrame({'column':train_variables, 'values':model.best_estimator_.coef_})
        #feature_importance.sort_values(by='values', axis=0, ascending=False, inplace=True)
        report.append([trainS, testS, f1S, preS,recS,AUROC,model.best_params_, i])
        #              feature_importance.iloc[0].values,feature_importance.iloc[1].values,feature_importance.iloc[2].values,
        #              feature_importance.iloc[3].values,feature_importance.iloc[4].values])
        print(i,':')
        print(trainS,'-',testS,'-',f1S,'-',preS,'-',recS,'-',AUROC,'-',model.best_params_)
        print('---------------------------------')
        res=pd.DataFrame(report)
        #res.to_csv('result/trained models/LR_models/report.csv')
        #joblib.dump(model, 'result/trained models/LR_models/'+i+'_LR.model')

In [ ]:
# random forest

report=[]
count=0
for time,scale_pos_weight in enumerate([0.01,0.02,0.03,0.04,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80
                                        ,90,100,200,300,400,500]+list2):
    for i in outcome_variables:
        count=count+1
        parameters=[{'n_estimators':[30,60],
                    'max_depth':[30,60],
                    'max_leaf_nodes':[30,60],
                    'class_weight':[scale_pos_weight]}]
        model=GridSearchCV(RandomForestClassifier(), parameters, cv=5)
        model.fit(train[train_variables], train[i])
        y_score=model.predict_proba(test[train_variables])[:,1]
        fpr, tpr, _ = roc_curve(test[i], y_score)
        AUROC=auc(fpr, tpr)
        trainS=model.score(train[train_variables], train[i])
        testS=model.score(test[train_variables], test[i])
        f1S=metrics.f1_score(test[i], model.predict(test[train_variables]))
        preS=metrics.precision_score(test[i], model.predict(test[train_variables]))
        recS=metrics.recall_score(test[i], model.predict(test[train_variables]))
        AUROC=auc(fpr, tpr)
        #feature_importance=pd.DataFrame({'column':train_variables, 'values':model.best_estimator_.coef_})
        #feature_importance.sort_values(by='values', axis=0, ascending=False, inplace=True)
        report.append([trainS, testS, f1S, preS,recS,AUROC,model.best_params_, i])
        #              feature_importance.iloc[0].values,feature_importance.iloc[1].values,feature_importance.iloc[2].values,
        #              feature_importance.iloc[3].values,feature_importance.iloc[4].values])
        #print(i,':')
        #print(trainS,'-',testS,'-',f1S,'-',preS,'-',recS,'-',AUROC,'-',model.best_params_)
        #print('---------------------------------')
    res=pd.DataFrame(report)
    #res.to_csv('result/trained models/RF_models/RF_report.csv')
        #joblib.dump(model, 'result/trained models/RF_models/'+i+'_RF.model')

In [ ]:
#artificial neural network

report=[]
count=0
for i in outcome_variables:
    count=count+1
    parameters=[{'hidden_layer_sizes':[(50,50),(100,100),(150,150)]}]
    #model=GridSearchCV(RandomForestClassifier(), parameters, cv=5)
    model=GridSearchCV(MLPClassifier(), parameters, cv=5)
    #model=RandomForestClassifier(class_weight='balanced')
    model.fit(train[train_variables], train[i])

    y_score=model.predict_proba(test[train_variables])[:,1]
    fpr, tpr, _ = roc_curve(test[i], y_score)
    AUROC=auc(fpr, tpr)
    trainS=model.score(train[train_variables], train[i])
    testS=model.score(test[train_variables], test[i])
    f1S=metrics.f1_score(test[i], model.predict(test[train_variables]))
    preS=metrics.precision_score(test[i], model.predict(test[train_variables]))
    recS=metrics.recall_score(test[i], model.predict(test[train_variables]))
    AUROC=auc(fpr, tpr)
    #feature_importance=pd.DataFrame({'column':train_variables, 'values':model.best_estimator_.coef_})
    #feature_importance.sort_values(by='values', axis=0, ascending=False, inplace=True)
    report.append([trainS, testS, f1S, preS,recS,AUROC,model.best_params_, i])
    #              feature_importance.iloc[0].values,feature_importance.iloc[1].values,feature_importance.iloc[2].values,
    #              feature_importance.iloc[3].values,feature_importance.iloc[4].values])
    #print(i,':')
    #print(trainS,'-',testS,'-',f1S,'-',preS,'-',recS,'-',AUROC,'-',model.best_params_)
    #print('---------------------------------')
    res=pd.DataFrame(report)
    res.to_csv('result/trained models/ANN_models/report.csv')
    joblib.dump(model, 'result/trained models/ANN_models/'+i+'_ANN.model')

In [ ]:
#ROC curve

name=['AKF','CKD','hospital mortality','inflammatory','pneumonia','sepsis','septic shock','Thrombocytopenia']
test_name=['AKF.csv','CKD.csv','hospital_mortality', 'inflammatory reaction.csv','pneumonia.csv',
         'sepsis.csv','septic shock.csv','Thrombocytopenia.csv']

for i in range(len(name)):
    plt.figure(dpi=500)
    modelXGB=joblib.load('result/trained models/XGB_models/'+test_name[i]+'_XGB.model')
    y_score=modelXGB.predict_proba(test[train_variables])[:,1]
    fpr, tpr, _ = roc_curve(test[test_name[i]], y_score)
    AUROC=auc(fpr, tpr)
    plt.plot(fpr, tpr, color='blue', label='XGB (AUROC=%0.2f)'%AUROC)
    modelANN=joblib.load('result/trained models/ANN_models/'+test_name[i]+'_ANN.model')
    y_score=modelANN.predict_proba(test[train_variables])[:,1]
    fpr, tpr, _ = roc_curve(test[test_name[i]], y_score)
    AUROC=auc(fpr, tpr)
    plt.plot(fpr, tpr, color='darkorange', label='ANN (AUROC=%0.2f)'%AUROC)
    modelLR=joblib.load('result/trained models/LR_models/'+test_name[i]+'_LR.model')
    y_score=modelLR.predict_proba(test[train_variables])[:,1]
    fpr, tpr, _ = roc_curve(test[test_name[i]], y_score)
    AUROC=auc(fpr, tpr)
    plt.plot(fpr, tpr, color='red', label='LR (AUROC=%0.2f)'%AUROC)
    modelRF=joblib.load('result/trained models/RF_models/'+test_name[i]+'_RF.model')
    y_score=modelRF.predict_proba(test[train_variables])[:,1]
    fpr, tpr, _ = roc_curve(test[test_name[i]], y_score)
    AUROC=auc(fpr, tpr)
    plt.plot(fpr, tpr, color='green', label='RF (AUROC=%0.2f)'%AUROC)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.title('ROC curve of '+name[i]+' model')
    plt.legend(loc="lower right")
    plt.savefig('result/ROC curve/'+name[i]+'.jpg')
    plt.show()

In [ ]:
#score table

model_name=['XGB','ANN','LR','RF']
res=[]
for i in range(len(test_name)):
    modelXGB=joblib.load('result/trained models/XGB_models/'+test_name[i]+'_XGB.model')
    modelANN=joblib.load('result/trained models/ANN_models/'+test_name[i]+'_ANN.model')
    modelLR=joblib.load('result/trained models/LR_models/'+test_name[i]+'_LR.model')
    modelRF=joblib.load('result/trained models/RF_models/'+test_name[i]+'_RF.model')
    model_list=[modelXGB,modelANN,modelLR,modelRF]
    for j in range(len(model_list)):
        y_score=model_list[j].predict_proba(test[train_variables])[:,1]
        fpr, tpr, _ = roc_curve(test[test_name[i]], y_score)
        trainS=model_list[j].score(train[train_variables], train[test_name[i]])
        testS=model_list[j].score(test[train_variables], test[test_name[i]])
        f1S=metrics.f1_score(test[test_name[i]], model_list[j].predict(test[train_variables]))
        preS=metrics.precision_score(test[test_name[i]], model_list[j].predict(test[train_variables]))
        recS=metrics.recall_score(test[test_name[i]], model_list[j].predict(test[train_variables]))
        AUROC=auc(fpr, tpr)
        #print(trainS,'-',testS,'-',f1S,'-',preS,'-',recS,'-',AUROC)
        #print(metrics.confusion_matrix(test[i], model.predict(test[train_variables])))
        res.append([trainS,testS,f1S,preS,recS,AUROC,name[i],model_name[j]])  
res=pd.DataFrame(res)
res.to_csv('result/ROC curve/score table.csv')


In [ ]:
# XGBoost
report=[]
count=0

for time,scale_pos_weight in enumerate([30]):
    for i in outcome_variables:
        count=count+1
        parameters=[{'n_estimators': [60], 
                    'max_depth': [30],
                    'reg_lambda':[10],
                    'gamma':[0.1],
                    'scale_pos_weight':[scale_pos_weight]}]
        model=GridSearchCV(XGBClassifier(n_jobs=-1), parameters, cv=5)
        model.fit(train[train_variables], train[i])
        y_score=model.predict_proba(test[train_variables])[:,1]
        fpr, tpr, _ = roc_curve(test[i], y_score)
        AUROC=auc(fpr, tpr)
        trainS=model.score(train[train_variables], train[i])
        testS=model.score(test[train_variables], test[i])
        f1S=metrics.f1_score(test[i], model.predict(test[train_variables]))
        preS=metrics.precision_score(test[i], model.predict(test[train_variables]))
        recS=metrics.recall_score(test[i], model.predict(test[train_variables]))
        AUROC=auc(fpr, tpr)
        feature_importance=pd.DataFrame({'column':train_variables, 'values':model.best_estimator_.feature_importances_})
        feature_importance.sort_values(by='values', axis=0, ascending=False, inplace=True)
        report.append([trainS, testS, f1S, preS,recS,AUROC,model.best_params_, i,
                      feature_importance.iloc[0].values,feature_importance.iloc[1].values,feature_importance.iloc[2].values,
                      feature_importance.iloc[3].values,feature_importance.iloc[4].values,scale_pos_weight])
        print(i,':')
        print('scale:', scale_pos_weight)
        print(trainS,'-',testS,'-',f1S,'-',preS,'-',recS,'-',AUROC,'-',model.best_params_)
        print('---------------------------------')
    res=pd.DataFrame(report)
    #res.to_csv('result/trained models/XGB_models/report.csv')
    joblib.dump(model, 'result/trained models/XGB_models/'+i+'_XGB.model')

In [ ]:
file_name=['test/hospital_mortality_XGB.model','test/liver_dysfunction_XGB.model',
          'test/septic_shock_XGB.model','test/thrombocytopenia_XGB.model']
y_name=['hospital_mortality','liver_dysfunction','septic_shock','thrombocytopenia']

for i in range(4):
    model=joblib.load(file_name[i])
    y_score=model.predict_proba(test[train_variables])[:,1]
    fpr, tpr, _ = roc_curve(test[y_name[i]], y_score)
    AUROC=auc(fpr, tpr)
    trainS=model.score(train[train_variables], train[y_name[i]])
    testS=model.score(test[train_variables], test[y_name[i]])
    f1S=metrics.f1_score(test[y_name[i]], model.predict(test[train_variables]))
    preS=metrics.precision_score(test[y_name[i]], model.predict(test[train_variables]))
    recS=metrics.recall_score(test[y_name[i]], model.predict(test[train_variables]))
    AUROC=auc(fpr, tpr)
    print(y_name[i],':',trainS,'-',testS,'-',f1S,'-',preS,'-',recS,'-',AUROC,'-',model.best_params_)
    print()

In [ ]:
model=joblib.load('result/trained models/XGB_models/hospital_mortality_XGB.model')

In [ ]:
model.best_estimator_.feature_importances_

In [ ]:
feature_importance=pd.DataFrame({'column':train_variables, 'values':model.best_estimator_.feature_importances_})
feature_importance.sort_values(by='values', axis=0, ascending=False, inplace=True)

In [ ]:
feature_importance